# Finite Strain 
[![Google Collab Book](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meyer-nils/torch-fem/blob/main/examples/basic/solid/finite_strain.ipynb)

We compare small-strain and finite-strain material models for a cantilever beam subjected to a concentrated force at its tip.

In [1]:
import torch
import pyvista

from torchfem import Solid
from torchfem.mesh import cube_hexa
from torchfem.materials import (
    IsotropicElasticity3D,
    IsotropicSaintVenantKirchhoff3D,
    NeoHookean3D,
)

# Set default data type to double precision
torch.set_default_dtype(torch.float64)

# Generate beam
L = 5.0
P = -5
nodes, elements = cube_hexa(21, 5, 5, L, 1.0, 1.0)
tip = (nodes[:, 0] == L) & (nodes[:, 1] == 0.5) & (nodes[:, 2] == 0.5)

print("Number of nodes:", nodes.shape[0])
print("Number of elements:", elements.shape[0])

Number of nodes: 525
Number of elements: 320


## Small strain isotropic elasticity

In [2]:
material = IsotropicElasticity3D(E=1000.0, nu=0.3)
beam1 = Solid(nodes, elements, material)

# Assign boundary conditions
beam1.constraints[nodes[:, 0] == 0, :] = True
beam1.forces[tip, 2] = P

# Solve
u1, f1, σ1, F1, α1 = beam1.solve()

print("Tip deflection (ABAQUS reference with 1 increment and 1 iteration):")
print(f"    X: {u1[tip,0].item():.3f} ( 0.000)")
print(f"    Y: {u1[tip,1].item():.3f} ( 0.000)")
print(f"    Z: {u1[tip,2].item():.3f} (-2.549)")
print(f"Maximum stress σ_xx: {σ1[:,0,0].max():.3f} (112.96)")
print(f"Maximum stress σ_xx: {σ1[:,0,0].min():.3f} (-112.96)")

Tip deflection (ABAQUS reference with 1 increment and 1 iteration):
    X: -0.000 ( 0.000)
    Y: -0.000 ( 0.000)
    Z: -2.466 (-2.549)
Maximum stress σ_xx: 108.787 (112.96)
Maximum stress σ_xx: -108.787 (-112.96)


## Isotropic Stain Venant-Kirchhoff

In [3]:
# Create model with Kirchhoff material
material = IsotropicSaintVenantKirchhoff3D(E=1000.0, nu=0.3)
beam2 = Solid(nodes, elements, material)

# Assign boundary conditions
beam2.constraints[nodes[:, 0] == 0, :] = True
beam2.forces[tip, 2] = P

# Solve
u2, f2, σ2, F2, α2 = beam2.solve(max_iter=100)

print("Tip deflection (ABAQUS reference with 1 increment and 15 iterations):")
print(f"    X: {u2[tip,0].item():.3f} (-0.506)")
print(f"    Y: {u2[tip,1].item():.3f} ( 0.000)")
print(f"    Z: {u2[tip,2].item():.3f} (-2.047)")
print(f"Maximum stress σ_xx: {σ2[:,0,0].max():.3f} (  97.325)")
print(f"Minimum stress σ_xx: {σ2[:,0,0].min():.3f} (-104.776)")

Tip deflection (ABAQUS reference with 1 increment and 15 iterations):
    X: -0.759 (-0.506)
    Y: -0.000 ( 0.000)
    Z: -2.351 (-2.047)
Maximum stress σ_xx: 118.633 (  97.325)
Minimum stress σ_xx: -99.316 (-104.776)


## Neo-Hookean 

In [4]:
# Create model with Neo-Hookean material
material = NeoHookean3D(lbd0=576.9231, mu0=384.6154)
# print(f"C10: {material.mu0/2}")
# print(f"D1: {2/(material.lbd0+2/3*material.mu0)}")
beam3 = Solid(nodes, elements, material)

# Assign boundary conditions
beam3.constraints[nodes[:, 0] == 0, :] = True
beam3.forces[tip, 2] = P

# Solve
u3, f3, σ3, F3, α3 = beam3.solve(max_iter=100)

print("Tip deflection (ABAQUS reference with 4 increments and 13 iterations):")
print(f"    X: {u3[tip,0].item():.3f} (-0.514)")
print(f"    Y: {u3[tip,1].item():.3f} ( 0.000)")
print(f"    Z: {u3[tip,2].item():.3f} (-2.089)")
print(f"Maximum stress σ_xx: {σ3[:,0,0].max():.3f} ( 97.937)")
print(f"Minimum stress σ_xx: {σ3[:,0,0].min():.3f} (-98.494)")

Tip deflection (ABAQUS reference with 4 increments and 13 iterations):
    X: -0.698 (-0.514)
    Y: -0.000 ( 0.000)
    Z: -2.309 (-2.089)
Maximum stress σ_xx: 106.444 ( 97.937)
Minimum stress σ_xx: -111.137 (-98.494)


## Comparison

In [5]:
plotter = pyvista.Plotter()
beam1.plot(color="gray", plotter=plotter, opacity=0.1)
beam1.plot(u=u1, color="cyan", opacity=0.8, plotter=plotter)
beam2.plot(u=u2, color="orange", opacity=0.8, plotter=plotter)
beam3.plot(u=u3, color="red", opacity=0.8, plotter=plotter)
plotter.show(jupyter_backend="html")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…